In [1]:
!pip install mediapipe-model-maker

In [2]:
!pip install tensorflow[and-cuda]

In [3]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2025-07-19 10:51:14.588036: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-19 10:51:14.588093: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-19 10:51:14.589767: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.15.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [4]:
!nvidia-smi

Sat Jul 19 10:51:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
from mediapipe_model_maker import object_detector
from mediapipe_model_maker import quantization
import pandas as pd

/usr/local/lib/python3.11/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [6]:
train_dataset_path = "/kaggle/input/pascal-voc-trash-detection-dataset/pascal_voc_trash_detection/train"
validation_dataset_path = "/kaggle/input/pascal-voc-trash-detection-dataset/pascal_voc_trash_detection/validation"

In [7]:
train_data = object_detector.Dataset.from_pascal_voc_folder(
    train_dataset_path,
    cache_dir="/kaggle/working/tmp/train",
)

val_data = object_detector.Dataset.from_pascal_voc_folder(
    validation_dataset_path,
    cache_dir="/kaggle/working/tmp/validation")

print("train_data size: ", train_data.size)
print("val_data size: ", val_data.size)

train_data size:  3442
val_data size:  860


In [8]:
#Model hyperparameters
BATCH_SIZE = 16
LEARNING_RATE = 0.1
EPOCHS = 80

#Model
MOBILENET_V2_I320 = object_detector.SupportedModels.MOBILENET_V2_I320

In [9]:
hparams = object_detector.HParams(batch_size=BATCH_SIZE, learning_rate=LEARNING_RATE, epochs=EPOCHS, export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=MOBILENET_V2_I320,
    hparams=hparams
)
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=val_data,
    options=options)

/usr/local/lib/python3.11/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 80, 80, 24   2257984   
                             ),                                  
                              '3': (None, 40, 40, 32             
                             ),                                  
                              '4': (None, 20, 20, 96             
                             ),                                  
                              '5': (None, 10, 10, 32             
                             0),                                 
                              '6': (None, 10, 10, 12             
                             80)}                                
                                                                 
 fpn (FPN)                   {'5': (None, 10, 10, 12   130880    
                             8),                  

/usr/local/lib/python3.11/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(
I0000 00:00:1752922536.289825     230 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


215/215 [==============================] - 148s 475ms/step - total_loss: 2.3562 - cls_loss: 2.1636 - box_loss: 0.0027 - model_loss: 2.2994 - val_total_loss: 1.5122 - val_cls_loss: 1.3777 - val_box_loss: 0.0016 - val_model_loss: 1.4554
Epoch 2/80
215/215 [==============================] - 79s 368ms/step - total_loss: 1.4790 - cls_loss: 1.3349 - box_loss: 0.0017 - model_loss: 1.4222 - val_total_loss: 1.4023 - val_cls_loss: 1.2851 - val_box_loss: 0.0012 - val_model_loss: 1.3455
Epoch 3/80
215/215 [==============================] - 80s 369ms/step - total_loss: 1.3755 - cls_loss: 1.2487 - box_loss: 0.0014 - model_loss: 1.3187 - val_total_loss: 1.3194 - val_cls_loss: 1.2100 - val_box_loss: 0.0011 - val_model_loss: 1.2627
Epoch 4/80
215/215 [==============================] - 79s 368ms/step - total_loss: 1.2807 - cls_loss: 1.1620 - box_loss: 0.0012 - model_loss: 1.2239 - val_total_loss: 1.1993 - val_cls_loss: 1.0957 - val_box_loss: 9.3636e-04 - val_model_loss: 1.1426
Epoch 5/80
215/215 [======

In [10]:
def save_model_history_to_csv(csv_filename="model_history.csv"):
    history_dict = model._history.history
    df_history = pd.DataFrame(history_dict)
    df_history.to_csv(csv_filename, index=True)

save_model_history_to_csv()

In [14]:
model.evaluate(val_data, batch_size=8)

108/108 [==============================] - 10s 74ms/step - total_loss: 0.2163 - cls_loss: 0.1366 - box_loss: 5.1319e-04 - model_loss: 0.1623


/usr/local/lib/python3.11/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=5.06s).
Accumulating evaluation results...
DONE (t=1.44s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.847
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.986
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.969
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.684
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.852
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.602
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.879
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.879
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 

([0.21627463400363922,
  0.13662993907928467,
  0.0005131890648044646,
  0.16228948533535004],
 {'AP': 0.8466386,
  'AP50': 0.9863021,
  'AP75': 0.9689669,
  'APs': -1.0,
  'APm': 0.6838392,
  'APl': 0.8520662,
  'ARmax1': 0.6024674,
  'ARmax10': 0.8790173,
  'ARmax100': 0.8790173,
  'ARs': -1.0,
  'ARm': 0.7698967,
  'ARl': 0.8821844})

In [15]:
model.export_model('model_pemilah_sampah.tflite')

Exporting a floating point model


/usr/local/lib/python3.11/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
Summary on the non-converted ops:
---------------------------------
 * Accepted dialects: tfl, builtin, func
 * Non-Converted Ops: 211, Total Ops 386, % non-converted = 54.66 %
 * 211 ARITH ops

- arith.constant:  211 occurrences  (f32: 207, i32: 4)



  (f32: 13)
  (f32: 2)
  (f32: 81)
  (f32: 64)
  (f32: 1)
  (f32: 1)
  (f32: 8)
  (f32: 2)


In [16]:
quantization_config = quantization.QuantizationConfig.for_float16()
model.export_model(model_name="model_pemilah_sampah_fp16.tflite", quantization_config=quantization_config)

Exporting with custom post-training-quantization:  <mediapipe_model_maker.python.core.utils.quantization.QuantizationConfig object at 0x7f8ffeecc9d0>


/usr/local/lib/python3.11/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/usr/local/lib/python3.11/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
Summary on the non-converted ops:
---------------------------------
 * Accepted dialects: tfl, builtin, func
 * Non-Converted Ops: 211, Total Ops 593, % non-converted = 35.58 %
 * 211 ARITH ops

- arith.constant:  211 occurrences  (f16: 207, i32: 4)



  (f32: 13)
  (f32: 2)
  (f32: 81)
  (f32: 64)
  (f32: 207)
  (f32: 1)
  (f32: 1)
  (f32: 8)
  (f32: 2)


In [19]:
new_qat_hparams = object_detector.QATHParams(learning_rate=0.1, batch_size=16, epochs=20)
model.restore_float_ckpt()
model.quantization_aware_training(train_data, val_data, qat_hparams=new_qat_hparams)

Using existing files at /tmp/model_maker/object_detector/mobilenetv2_i320
Model: "retina_net_model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net_2 (MobileNet)    {'2': (None, 80, 80, 24   2257984   
                             ),                                  
                              '3': (None, 40, 40, 32             
                             ),                                  
                              '4': (None, 20, 20, 96             
                             ),                                  
                              '5': (None, 10, 10, 32             
                             0),                                 
                              '6': (None, 10, 10, 12             
                             80)}                                
                                                                 
 fpn_2 (FPN)                 {'5': (None

/usr/local/lib/python3.11/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


215/215 [==============================] - 242s 872ms/step - total_loss: 1.3483 - cls_loss: 1.0300 - box_loss: 0.0064 - model_loss: 1.3483 - val_total_loss: 1.3691 - val_cls_loss: 1.1807 - val_box_loss: 0.0038 - val_model_loss: 1.3691
Epoch 2/20
215/215 [==============================] - 162s 750ms/step - total_loss: 0.8101 - cls_loss: 0.6765 - box_loss: 0.0027 - model_loss: 0.8101 - val_total_loss: 0.7464 - val_cls_loss: 0.6396 - val_box_loss: 0.0021 - val_model_loss: 0.7464
Epoch 3/20
215/215 [==============================] - 162s 753ms/step - total_loss: 0.7455 - cls_loss: 0.6236 - box_loss: 0.0024 - model_loss: 0.7455 - val_total_loss: 0.6543 - val_cls_loss: 0.5541 - val_box_loss: 0.0020 - val_model_loss: 0.6543
Epoch 4/20
215/215 [==============================] - 162s 751ms/step - total_loss: 0.7298 - cls_loss: 0.6084 - box_loss: 0.0024 - model_loss: 0.7298 - val_total_loss: 0.6454 - val_cls_loss: 0.5455 - val_box_loss: 0.0020 - val_model_loss: 0.6454
Epoch 5/20
215/215 [=======

In [20]:
model.export_model('model_pemilah_sampah_int8.tflite')

Exporting a qat int8 model


/usr/local/lib/python3.11/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
Summary on the non-converted ops:
---------------------------------
 * Accepted dialects: tfl, builtin, func
 * Non-Converted Ops: 4, Total Ops 446, % non-converted = 0.90 %
 * 4 ARITH ops

- arith.constant:    4 occurrences  (i32: 4)



  (uq_8: 12)
  (uq_8: 2)
  (uq_8: 81)
  (uq_8: 68)
  (f32: 2)
  (uq_8: 1)
  (uq_8: 113, uq_32: 149)
  (uq_8: 1)
  (uq_8: 8)
  (uq_8: 2)


In [17]:
!zip -r exported_model.zip /kaggle/working/exported_model

updating: kaggle/working/exported_model/ (stored 0%)
updating: kaggle/working/exported_model/model_pemilah_sampah_fp16.tflite (deflated 15%)
updating: kaggle/working/exported_model/float_ckpt.data-00000-of-00001 (deflated 8%)
updating: kaggle/working/exported_model/model_pemilah_sampah.tflite (deflated 11%)
updating: kaggle/working/exported_model/float_ckpt.index (deflated 81%)
updating: kaggle/working/exported_model/metadata.json (deflated 98%)
